#Notebook Silver --> Gold

In [0]:
# Importação de bibliotecas
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import timedelta
from pyspark.sql.types import DecimalType

In [0]:
# Definição do caminho da camada Gold
catalogo = "medalhao_case"
gold_db_name = "gold"

In [0]:
# Leitura das tabelas Silver
silver_base_atendentes_df = spark.table("medalhao_case.silver.silver_base_atendentes")
silver_base_canais_df = spark.table("medalhao_case.silver.silver_base_canais")
silver_base_motivos_df = spark.table("medalhao_case.silver.silver_base_motivos")
silver_chamados_df = spark.table("medalhao_case.silver.silver_chamados")
silver_chamados_hora_df = spark.table("medalhao_case.silver.silver_chamados_hora")
silver_clientes_df = spark.table("medalhao_case.silver.silver_clientes")
silver_custos_df = spark.table("medalhao_case.silver.silver_custos")
silver_pesquisa_satisfacao_df = spark.table("medalhao_case.silver.silver_pesquisa_satisfacao")

## Tabelas : Fato e dimesão

### Tabela  gold.ft_chamados_atendimento

In [0]:
ft_chamados_atendimento_df = (
    silver_chamados_df
    
    # Join com clientes
    .join(silver_clientes_df, "id_cliente", "left")
    
    # Join com motivos (campo 'motivo' da tabela de chamados = nome_motivo da base de motivos)
    .join(silver_base_motivos_df, F.col("motivo") == F.col("nome_motivo"), "left")
    
    # Join com canais (campo 'canal' da tabela de chamados = nome_canal da base de canais)
    .join(silver_base_canais_df, F.col("canal") == F.col("nome_canal"), "left")
    
    # Join com atendentes (id_atendente)
    # Deve ser necessariamente um left join, pois existem chamados sem id_atendente
    .join(silver_base_atendentes_df, "id_atendente", "left")
    
    # Join com custos
    .join(silver_custos_df, "id_chamado", "left")
    
    # Join com pesquisa de satisfação
    .join(silver_pesquisa_satisfacao_df, "id_chamado", "left")
    
    
    # Selecionar colunas
    .select(
        "id_chamado",
        # fk_tempo_abertura
        F.format_string(
            "%d%02d%02d%02d",
            F.year("hora_abertura_chamado"),
            F.month("hora_abertura_chamado"),
            F.dayofmonth("hora_abertura_chamado"),
            F.hour("hora_abertura_chamado")
        ).cast("long").alias("fk_tempo_abertura"),
        F.col("id_cliente").alias("fk_cliente"),
        F.col("id_motivo").alias("fk_motivo"),
        F.col("nome_canal").alias("fk_canal"),
        F.col("id_atendente").alias("fk_atendente"),
        F.col("hora_inicio_atendimento"),
        F.col("hora_finalizacao_atendimento"),
        F.col("tempo_espera").alias("tempo_espera_seg"),
        F.col("tempo_atendimento").alias("tempo_atendimento_seg"),
        F.col("custo").alias("custo_atendimento"),
        F.col("nota_atendimento"),
        F.col("resolvido").alias("atendimento_resolvido")
    
    )
)

ft_chamados_atendimento_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalogo}.{gold_db_name}.ft_chamados_atendimento")


### Tabela gold.dim_atendente

In [0]:
dim_atendente_df =(
    silver_base_atendentes_df
    .select(
        F.col("id_atendente"),
        F.col("nome_atendente"),
        F.when(F.col("nivel_atendimento") == 1, "NÍVEL 1")
        .when(F.col("nivel_atendimento") == 2, "NÍVEL 2").alias("nivel_atendimento")
    )
)
display(dim_atendente_df)
dim_atendente_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalogo}.{gold_db_name}.dim_atendente")


### Tabela gold.dim_cliente

In [0]:
dim_cliente_df = ( silver_clientes_df
                  .select(
                      F.col("id_cliente").alias("id_cliente"),
                      F.col("nome").alias("nome_cliente"),
                      F.col("regiao").alias("regiao_cliente"),
                      F.col("idade").alias("idade_cliente")
                      )
                  .withColumn( "faixa_etaria", 
                              F.when(F.col("idade_cliente").isNull(), "Não informada")
                              .when(F.col("idade_cliente") < 18, "Menor de Idade")
                              .when( (F.col("idade_cliente") >= 18) & (F.col("idade_cliente") < 30), "Jovem")
                              .when( (F.col("idade_cliente") >= 30) & (F.col("idade_cliente") <= 60), "Adulto")
                              .otherwise("Idoso")
                            )
                )
                
dim_cliente_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalogo}.{gold_db_name}.dim_cliente")


### Tabela gold.dim_motivo

In [0]:
# Leitura da tabela motivos na camada silver
silver_motivos_df = spark.table("medalhao_case.silver.silver_base_motivos")

dim_motivo = (
    silver_motivos_df
    .select(
        # Selecionei apenas as colunas que serão usadas na dimensão
        F.col("id_motivo").cast('int').alias("id_motivo"),
        F.col("nome_motivo"),
        F.col("categoria"),
        F.col("criticidade"))

        # Colunas para elencar os motivos de autosserviço e especialista
        .withColumn(
            "motivo_de_autosservico",
            F.when(
                F.col("criticidade") == "Baixa", "Sim"
            ).otherwise("Não")
        )
        .withColumn(
            "motivo_de_especialista",
            F.when(
                F.col("criticidade") == "Alta", "Sim"
            ).otherwise("Não")
        )
    )

#Escrita na camada gold
dim_motivo.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalogo}.{gold_db_name}.dim_motivo")

### Tabela gold.dim_canal 

In [0]:

dim_canal = (
  silver_base_canais_df.select('nome_canal','status_canal')

  #
  .withColumn('tipo_atendimento',
    F.when(F.col('nome_canal').ilike('atendimento%'),'NÃO AUTOMATIZADO').otherwise('AUTOMATIZADO')
  )
)

dim_canal.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalogo}.{gold_db_name}.dim_canal") 

### Tabela gold.dim_tempo 

In [0]:
data_inicio = "2024-01-01"
data_fim = "2026-12-31"
df_tempo = (
    # Gerei sequência de dias
    spark.range(1)
    .select(F.explode(F.sequence(
        F.to_date(F.lit(data_inicio)),
        F.to_date(F.lit(data_fim)),
        F.expr("INTERVAL 1 DAY")
    )).alias("data"))
    
    # Extração de Atributos Básicos
    .withColumn("ano", F.year("data"))
    .withColumn("mes", F.month("data"))
    .withColumn("dia", F.dayofmonth("data"))
    .withColumn("dia_semana_num", F.dayofweek("data")) # 1=Domingo, 7=Sábado
    
    # Atributos de Texto
    .withColumn("nome_dia_semana", F.date_format("data", "EEEE")) 
    .withColumn("nome_mes", F.date_format("data", "MMMM"))      
    
    # Tradução
    .withColumn("nome_dia_semana", 
        F.when(F.col("nome_dia_semana") == "Monday", "Segunda-feira")
         .when(F.col("nome_dia_semana") == "Tuesday", "Terça-feira")
         .when(F.col("nome_dia_semana") == "Wednesday", "Quarta-feira")
         .when(F.col("nome_dia_semana") == "Thursday", "Quinta-feira")
         .when(F.col("nome_dia_semana") == "Friday", "Sexta-feira")
         .when(F.col("nome_dia_semana") == "Saturday", "Sábado")
         .when(F.col("nome_dia_semana") == "Sunday", "Domingo")
         .otherwise(F.col("nome_dia_semana"))
    )
    .withColumn("nome_mes", 
        F.when(F.col("nome_mes") == "January", "Janeiro")
         .when(F.col("nome_mes") == "February", "Fevereiro")
         .when(F.col("nome_mes") == "March", "Março")
         .when(F.col("nome_mes") == "April", "Abril")
         .when(F.col("nome_mes") == "May", "Maio")
         .when(F.col("nome_mes") == "June", "Junho")
         .when(F.col("nome_mes") == "July", "Julho")
         .when(F.col("nome_mes") == "August", "Agosto")
         .when(F.col("nome_mes") == "September", "Setembro")
         .when(F.col("nome_mes") == "October", "Outubro")
         .when(F.col("nome_mes") == "November", "Novembro")
         .when(F.col("nome_mes") == "December", "Dezembro")
         .otherwise(F.col("nome_mes"))
    )
    
    # Flag pra analisar fluxo da semana com o fim de semana depois
    .withColumn("fim_de_semana", 
        F.when(F.col("nome_dia_semana").isin("Sábado", "Domingo"), "Sim")
         .otherwise("Não")
    )
    
    # Trimestre e Semestre pode ser útil, talvez até bimestre talvez
    .withColumn("trimestre", F.quarter("data"))
    .withColumn("semestre", F.when(F.col("mes") <= 6, 1).otherwise(2))
)

# Adicionar Tabela de Horas/Turnos
# Fiz um DATA-HORA porque acho que seria o mais simples e eficiente mas podemos criar uma dimensão de DATA e outra de HORA.

df_horas = spark.range(0, 24).toDF("hora_do_dia")

# Cada dia terá 24 registros 
df_dim_tempo_completa = (
    df_tempo.crossJoin(df_horas)
    .withColumn("turno", 
        F.when((F.col("hora_do_dia") >= 0) & (F.col("hora_do_dia") < 6), "Madrugada")
         .when((F.col("hora_do_dia") >= 6) & (F.col("hora_do_dia") < 12), "Manhã")
         .when((F.col("hora_do_dia") >= 12) & (F.col("hora_do_dia") < 18), "Tarde")
         .otherwise("Noite")
    )
    # Essa chave substituta vai facilitar e otimizar na hora de cruzar com a fato
    # Formato: YYYYMMDDHH (Inteiro)
    .withColumn("sk_tempo", 
        F.format_string("%d%02d%02d%02d", F.col("ano"), F.col("mes"), F.col("dia"), F.col("hora_do_dia")).cast("long")
    )
)

df_dim_tempo_completa.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalogo}.{gold_db_name}.dim_tempo")

##Views:
- gold.view_funil_gargalo 
- gold.view_kpi_qualidade_cliente 
- gold.view_performance_por_demografia 
- gold.view_top_motivos_impacto
- gold.view_comparativo_atendente_motivo 

### gold.view_funil_gargalo 

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao_case.gold.view_funil_gargalo AS
WITH fato_atendente_canal AS (
    SELECT 
        f.id_chamado,
        f.custo_atendimento,
        f.atendimento_resolvido,
        CASE 
            WHEN c.tipo_atendimento = 'AUTOMATIZADO' THEN 'Autosserviço'
            WHEN c.tipo_atendimento = 'NÃO AUTOMATIZADO'
                    AND a.nivel_atendimento = 'NÍVEL 1' THEN 'NÍVEL 1'
            WHEN c.tipo_atendimento = 'NÃO AUTOMATIZADO'
                    AND a.nivel_atendimento = 'NÍVEL 2' THEN 'NÍVEL 2'
            ELSE 'Indefinido'
        END AS nivel_funil
    FROM medalhao_case.gold.ft_chamados_atendimento f
    LEFT JOIN medalhao_case.gold.dim_atendente a
        ON f.fk_atendente = a.id_atendente
    LEFT JOIN medalhao_case.gold.dim_canal c
        ON f.fk_canal = c.nome_canal
)

SELECT
    nivel_funil AS nivel,
    -- Não há ids de chamados repetidos
    COUNT(*) AS volume_chamados,
    AVG(custo_atendimento) AS custo_medio,
    SUM(custo_atendimento) AS custo_total,
    CASE 
        WHEN COUNT(*) = 0 THEN 0 
        ELSE SUM(CASE WHEN atendimento_resolvido = 'Sim' THEN 1 ELSE 0 END) * 1.0 / COUNT(*)
    END AS perc_resolucao
FROM fato_atendente_canal
GROUP BY nivel_funil;

SELECT *
FROM medalhao_case.gold.view_funil_gargalo;

### gold.view_efetividade

In [0]:
%sql
create or replace view medalhao_case.gold.view_efetividade_canal as
select 
  fk_canal,
  count('*') as qte_chamados,
  sum(
    case when atendimento_resolvido = 'Sim' then 1 else 0 end
  ) as qte_resolvido,
  sum(
    case when atendimento_resolvido = 'Não' then 1 else 0 end
  ) as qte_nao_resolvido,
  round(avg(nota_atendimento)) as nota_media
from medalhao_case.gold.ft_chamados_atendimento
group by fk_canal


In [0]:
%sql
create or replace view medalhao_case.gold.view_efetividade_motivo as
select 
  fk_motivo,nome_motivo,
  count('*') as qte_chamados,
  sum(
    case when atendimento_resolvido = 'Sim' then 1 else 0 end
  ) as qte_resolvido,
  sum(
    case when atendimento_resolvido = 'Não' then 1 else 0 end
  ) as qte_nao_resolvido,
  round(100*qte_resolvido / qte_chamados,2) as efetividade
from medalhao_case.gold.ft_chamados_atendimento
left join medalhao_case.gold.dim_motivo
on medalhao_case.gold.ft_chamados_atendimento.fk_motivo = medalhao_case.gold.dim_motivo.id_motivo
group by fk_motivo,nome_motivo
order by fk_motivo


In [0]:
%sql
create or replace view medalhao_case.gold.view_efetividade_motivo_canal as
select 
  fk_motivo,nome_motivo,fk_canal,
  count('*') as qte_chamados,
  sum(
    case when atendimento_resolvido = 'Sim' then 1 else 0 end
  ) as qte_resolvido,
  sum(
    case when atendimento_resolvido = 'Não' then 1 else 0 end
  ) as qte_nao_resolvido,
  round(100*qte_resolvido / qte_chamados,2) as efetividade
from medalhao_case.gold.ft_chamados_atendimento
left join medalhao_case.gold.dim_motivo
on medalhao_case.gold.ft_chamados_atendimento.fk_motivo = medalhao_case.gold.dim_motivo.id_motivo
group by fk_motivo,nome_motivo,fk_canal
order by fk_motivo

In [0]:
%sql
create or replace view medalhao_case.gold.view_motivo_faixa_etaria as
select 
  faixa_etaria,nome_motivo,
  count('*') as qte_chamados,
  sum(
    case when atendimento_resolvido = 'Sim' then 1 else 0 end
  ) as qte_resolvido,
  sum(
    case when atendimento_resolvido = 'Não' then 1 else 0 end
  ) as qte_nao_resolvido,
  round(100*qte_resolvido / (qte_resolvido + qte_nao_resolvido),2) as efetividade,
  round(avg(coalesce(nota_atendimento,1)),2) as nota_media
from medalhao_case.gold.ft_chamados_atendimento

left join medalhao_case.gold.dim_motivo
on medalhao_case.gold.ft_chamados_atendimento.fk_motivo = medalhao_case.gold.dim_motivo.id_motivo

left join medalhao_case.gold.dim_cliente
on medalhao_case.gold.ft_chamados_atendimento.fk_cliente = medalhao_case.gold.dim_cliente.id_cliente

group by faixa_etaria,nome_motivo
order by faixa_etaria

### gold.view_kpi_qualidade_cliente

In [0]:
%sql
create or replace view medalhao_case.gold.view_kpi_qualidade_cliente as 
select
    c.id_cliente,
    c.nome_cliente,
    c.regiao_cliente,
    c.idade_cliente,
    c.faixa_etaria,

    count(*) as qtde_atendimentos,

    avg(f.tempo_espera_seg) as tempo_medio_espera_seg,
    avg(f.tempo_atendimento_seg) as tempo_medio_atendimento_seg,

    avg(f.nota_atendimento) as nota_media_satisfacao,

    avg(case when f.atendimento_resolvido = 'Sim' then 1.0 else 0.0 end) as taxa_resolucao,

    sum(f.custo_atendimento) as custo_total,
    avg(f.custo_atendimento) as custo_medio_por_chamado

from
    medalhao_case.gold.ft_chamados_atendimento f
    inner join medalhao_case.gold.dim_cliente c on f.fk_cliente = c.id_cliente

group by
    c.id_cliente,
    c.nome_cliente,
    c.regiao_cliente,
    c.idade_cliente,
    c.faixa_etaria;

select * from medalhao_case.gold.view_kpi_qualidade_cliente



###### Visualização para idosos

In [0]:
%sql
select *
from medalhao_case.gold.view_kpi_qualidade_cliente
where faixa_etaria = 'Idoso'
order by nota_media_satisfacao asc;


In [0]:
%sql
select
    count(case when nota_media_satisfacao is null then 1 end) as qtd_null,
    count(case when nota_media_satisfacao = 1 then 1 end) as qtd_nota_1,
    count(case when nota_media_satisfacao = 2 then 1 end) as qtd_nota_2,
    count(case when nota_media_satisfacao = 3 then 1 end) as qtd_nota_3,
    count(case when nota_media_satisfacao = 4 then 1 end) as qtd_nota_4,
    count(case when nota_media_satisfacao = 5 then 1 end) as qtd_nota_5

from medalhao_case.gold.view_kpi_qualidade_cliente
where faixa_etaria = 'Idoso';

###### Visualização para adultos

In [0]:
%sql
select *
from medalhao_case.gold.view_kpi_qualidade_cliente
where faixa_etaria = 'Adulto'
order by nota_media_satisfacao asc;


In [0]:
%sql
select
    count(case when nota_media_satisfacao is null then 1 end) as qtd_null,
    count(case when nota_media_satisfacao = 1 then 1 end) as qtd_nota_1,
    count(case when nota_media_satisfacao = 2 then 1 end) as qtd_nota_2,
    count(case when nota_media_satisfacao = 3 then 1 end) as qtd_nota_3,
    count(case when nota_media_satisfacao = 4 then 1 end) as qtd_nota_4,
    count(case when nota_media_satisfacao = 5 then 1 end) as qtd_nota_5

from medalhao_case.gold.view_kpi_qualidade_cliente
where faixa_etaria = 'Adulto';

###### Visualização para jovens

In [0]:
%sql
select *
from medalhao_case.gold.view_kpi_qualidade_cliente
where faixa_etaria = 'Jovem'
order by nota_media_satisfacao asc;


In [0]:
%sql
select
    count(case when nota_media_satisfacao is null then 1 end) as qtd_null,
    count(case when nota_media_satisfacao = 1 then 1 end) as qtd_nota_1,
    count(case when nota_media_satisfacao = 2 then 1 end) as qtd_nota_2,
    count(case when nota_media_satisfacao = 3 then 1 end) as qtd_nota_3,
    count(case when nota_media_satisfacao = 4 then 1 end) as qtd_nota_4,
    count(case when nota_media_satisfacao = 5 then 1 end) as qtd_nota_5

from medalhao_case.gold.view_kpi_qualidade_cliente
where faixa_etaria = 'Jovem';

###### Resolução média por canal e faixa etária

In [0]:
%sql
create or replace view medalhao_case.gold.view_eficiencia_canal_faixa_etaria as
select
    c.faixa_etaria as faixa_etaria_cliente,
    ca.nome_canal as canal,
    ca.tipo_atendimento as tipo_canal,
    
    -- Volume
    COUNT(f.id_chamado) as quantidade_chamados,
    
    -- Taxa de resolução
    round( sum(case when f.atendimento_resolvido = 'Sim' then 1 else 0 end) / count(f.id_chamado) * 100, 2
    ) as taxa_resolucao_percent,
    
    -- Tempos
    round(avg(f.tempo_espera_seg), 0) as tme_medio_seg,
    round(avg(f.tempo_atendimento_seg), 0) as tma_medio_seg,
    
    -- Satisfação
    round(avg(f.nota_atendimento), 2) as nota_media_satisfacao,
    
    -- Custo
    round(avg(f.custo_atendimento), 2) as custo_medio_chamado

from medalhao_case.gold.ft_chamados_atendimento f
inner join medalhao_case.gold.dim_cliente c on f.fk_cliente = c.id_cliente
inner join medalhao_case.gold.dim_canal ca on f.fk_canal = ca.nome_canal

group by
    c.faixa_etaria,
    ca.nome_canal,
    ca.tipo_atendimento

order by
    c.faixa_etaria,
    ca.nome_canal;

select *
from medalhao_case.gold.view_eficiencia_canal_faixa_etaria
order by faixa_etaria_cliente, canal


###### Recorrencia de chamados por faixa etária


In [0]:
%sql
create or replace view medalhao_case.gold.view_recorrencia_chamados_faixa_etaria as
with base_cliente as ( -- Calculo da quantidade de chamados por cliente
    select f.fk_cliente, count(distinct f.id_chamado) as qtde_chamados_cliente
    from medalhao_case.gold.ft_chamados_atendimento f
    group by f.fk_cliente
),

cliente_com_perfil as (
    select
        c.id_cliente,
        c.nome_cliente,
        c.regiao_cliente,
        c.faixa_etaria,

        coalesce(b.qtde_chamados_cliente, 0) as qtde_chamados_cliente,

        -- Flag (0/1) indicando se cliente abriu 2 ou mais chamados
        case 
            when coalesce(b.qtde_chamados_cliente, 0) >= 2 then 1
            else 0
        end as flag_recontato

    from medalhao_case.gold.dim_cliente c
    left join base_cliente b on c.id_cliente = b.fk_cliente
)

select
    faixa_etaria as faixa_etaria_cliente,

    count(*) as qtde_clientes,
    sum(qtde_chamados_cliente) as qtde_total_chamados,

    sum(flag_recontato) as qtde_clientes_recontato,

    -- Percentual de recontato
    round(sum(flag_recontato) * 100.0 / count(*), 2 ) as perc_prob_recontato,

    -- Média de chamados por cliente dentro de cada faixa etária
    round(avg(qtde_chamados_cliente), 2) as qtde_media_chamados_por_cliente

from cliente_com_perfil
group by faixa_etaria
order by 
  perc_prob_recontato desc, 
  faixa_etaria;

select *
from medalhao_case.gold.view_recorrencia_chamados_faixa_etaria
order by perc_prob_recontato desc, faixa_etaria_cliente;

In [0]:
%sql
create or replace view medalhao_case.gold.view_recorrencia_com_fcr as
select
    faixa_etaria_cliente,
    qtde_clientes,
    qtde_total_chamados,
    qtde_clientes_recontato,
    perc_prob_recontato,
    -- FCR = 100 - % recontato
    round(100 - perc_prob_recontato, 2) as fcr_percent
from medalhao_case.gold.view_recorrencia_chamados_faixa_etaria;

select *
from medalhao_case.gold.view_recorrencia_com_fcr;

### gold.view_performance_por_demografia 

In [0]:
%sql
create or replace view medalhao_case.gold.view_performance_por_demografia as
select
  c.regiao_cliente as regiao_cliente,
  c.faixa_etaria as faixa_etaria_cliente,
  -- Volume de Chamados
  count(f.id_chamado) as quantidade_chamados,
  -- Qualidade Percebida que achar pela média da satisfação
  round(avg(p.nota_atendimento),2) as nota_media,

  -- Eficiência Operacional (tempo de espera)
  round(avg(f.tempo_espera_seg)) as tme_medio_seg,
  
  -- Eficiência Operacional (tempo de duração),
  round(avg(f.tempo_atendimento_seg)) as tma_medio_seg,

  -- Eficiência Financeira (custo)
  round(avg(f.custo_atendimento), 2) as custo_medio_chamado,
  round(sum(f.custo_atendimento), 2) as custo_total,
  -- Resolutividade
  round(sum(case when f.atendimento_resolvido = 'Sim' then 1 else 0 end)/ count(f.id_chamado)*100, 1) as taxa_resolucao


from
  medalhao_case.gold.ft_chamados_atendimento as f
  inner join -- Para o perfil demográfico
    medalhao_case.gold.dim_cliente as c
    on f.fk_cliente = c.id_cliente
  left join -- Porque nem todos respondem a satisfação (nulos)
    medalhao_case.silver.silver_pesquisa_satisfacao as p
    on f.id_chamado = p.id_chamado
  left join -- Evitando chamados sem custo apurado
    medalhao_case.silver.silver_custos as k 
    on f.id_chamado = k.id_chamado

GROUP BY 
    c.regiao_cliente,
    c.faixa_etaria

ORDER BY 
    c.regiao_cliente,
    c.faixa_etaria;

select * from medalhao_case.gold.view_performance_por_demografia


In [0]:
%sql
SELECT 
    faixa_etaria_cliente,
    -- Volume para ver representatividade
    SUM(quantidade_chamados) AS total_chamados,
    -- Média ponderada de Custo
    ROUND(SUM(custo_medio_chamado * quantidade_chamados) / SUM(quantidade_chamados), 2) AS csat_geral,
    -- Média ponderada de TMA (Tempo de Atendimento)
    ROUND(SUM(tma_medio_seg * quantidade_chamados) / SUM(quantidade_chamados), 2) AS tma_geral_min,
    -- Custo Total acumulado
    ROUND(SUM(custo_total), 2) AS custo_total_faixa,
    -- Taxa de Resolução Média
    ROUND(AVG(taxa_resolucao), 1) AS taxa_resolucao_media
FROM 
    medalhao_case.gold.view_performance_por_demografia
GROUP BY 
    faixa_etaria_cliente
ORDER BY 
    custo_total_faixa DESC;

In [0]:
%sql
SELECT 
    regiao_cliente,
    SUM(quantidade_chamados) AS chamados,
    -- Tempo Médio de Espera Geral da Região
    ROUND(SUM(tme_medio_seg * quantidade_chamados) / SUM(quantidade_chamados), 0) AS tme_ponderado_seg,
    -- Satisfação Geral da Região
    ROUND(SUM(custo_medio_chamado * quantidade_chamados) / SUM(quantidade_chamados), 2) AS csat_ponderado,
    -- Custo Total da Região
    ROUND(SUM(custo_total), 2) AS custo_total_regional,
    -- Custo Médio Unitário
    ROUND(SUM(custo_total) / SUM(quantidade_chamados), 2) AS custo_medio_unitario
FROM 
    medalhao_case.gold.view_performance_por_demografia
GROUP BY 
    regiao_cliente
ORDER BY 
    tme_ponderado_seg DESC; -- Ordena por quem espera mais

### gold.view_top_motivos_impacto

In [0]:
%sql
create or replace view medalhao_case.gold.view_top_motivos_impacto as
with motivos_agregados as (
  -- Prmeiro fiz a união da dim de motivo com algumas colunas da fato
  select
    d.id_motivo,
    d.nome_motivo,
    d.categoria,
    d.criticidade,
    cast(coalesce(f.custo_atendimento, 0) as decimal(12,2)) as custo_atendimento,
    -- Conversão de "Sim" para 1 e "Não" para 0, para calculo da média
    case
      when atendimento_resolvido = 'Sim' then 1
      else 0
    end as atendimento_resolvido_binario
  from
    medalhao_case.gold.dim_motivo as d
  left join
    medalhao_case.gold.ft_chamados_atendimento as f
  on f.fk_motivo = d.id_motivo
),

kpi_custo_volume as (
  -- Agora a agregação pela categoria e o cálculo do custo total por motivo
  select
    categoria,
    nome_motivo,
    criticidade,
    count(id_motivo) as volume_motivo,
    sum(custo_atendimento) as custo_total_motivo,
    -- Uso da conversão do binário para calcular a porcentagem de resolução
    round(avg(atendimento_resolvido_binario) * 100, 2) as percentual_resolucao_motivo
  from
    motivos_agregados
  group by
    categoria, nome_motivo, criticidade
),

kpi_ranking_maior_custo as (
  -- Rankeia os motivos que possuem os maiores custos
  select
    *,
    rank() over(order by custo_total_motivo desc) as rank_custo_total_motivo
  from
    kpi_custo_volume
)

select
  -- Contexto e priorização
  categoria,
  nome_motivo,
  criticidade,
  -- KPIs
  volume_motivo,
  custo_total_motivo,
  percentual_resolucao_motivo,
  case 
    -- filtragem dos motivos que mais geram custos a empresa(entre os 14 do case)
    when rank_custo_total_motivo <= 2 then 'Urgente'
    when rank_custo_total_motivo <= 7 then 'Atenção'
    else 'Estável'
  end as top_custo_motivo
from
  kpi_ranking_maior_custo
order by
  custo_total_motivo desc;

SELECT * 
FROM medalhao_case.gold.view_top_motivos_impacto;

visualização da view anterior

In [0]:
%sql
-- Aqui existem 14 motivos desde a silver até a gold
SELECT 
    nome_motivo
FROM medalhao_case.silver.silver_base_motivos
GROUP BY nome_motivo;

In [0]:
%sql
-- mas na fato existem apenas 11 motivos desde a camada bronze até a gold tbm
SELECT 
    motivo
FROM medalhao_case.bronze.bronze_chamados
GROUP BY motivo;

In [0]:
%sql
select 
  categoria,
  nome_motivo,
  criticidade,
  volume_motivo,
  custo_total_motivo,
  percentual_resolucao_motivo,
  top_custo_motivo
from 
  medalhao_case.gold.view_top_motivos_impacto
order by
  custo_total_motivo desc;

-- Renegociação de dívida, Cancelamento de conta, Transferência de agência

### gold.view_comparativo_atendente_motivo 

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao_case.gold.view_comparativo_atendente_motivo AS
SELECT
  a.nome_atendente,
  a.nivel_atendimento, -- Diferencia Junior (N1) de Especialista (N2)
  m.nome_motivo,
  m.categoria AS categoria_motivo,
  m.criticidade, -- Mostra se o problema é Simples ou Complexo

  -- Métricas Calculadas
  COUNT(f.id_chamado) AS total_chamados,
  ROUND(AVG(f.tempo_atendimento_seg), 0) AS tma_medio_seg,
  ROUND(AVG(f.nota_atendimento), 2) AS media_nota_satisfacao,
  
  -- Cálculo da Taxa de Resolução (Assume que a coluna tem 'Sim' para resolvido)
  ROUND(
    SUM(CASE WHEN f.atendimento_resolvido = 'Sim' THEN 1 ELSE 0 END) 
    / COUNT(f.id_chamado) * 100
  , 1) AS taxa_resolucao_percentual,

  -- Soma dos Custos (Visão Financeira)
  ROUND(SUM(f.custo_atendimento), 2) AS custo_total_gerado

FROM
  medalhao_case.gold.ft_chamados_atendimento AS f
INNER JOIN
  medalhao_case.gold.dim_atendente AS a ON f.fk_atendente = a.id_atendente
INNER JOIN
  medalhao_case.gold.dim_motivo AS m ON f.fk_motivo = m.id_motivo

GROUP BY
  a.nome_atendente,
  a.nivel_atendimento,
  m.nome_motivo,
  m.categoria,
  m.criticidade

In [0]:
%sql
WITH classificacao AS (
  SELECT
    CASE
      -- 1. Caso Crítico: Nota Ruim E Demora Muito
      WHEN media_nota_satisfacao < 2.8 AND tma_medio_seg > 1200 THEN '1. CRÍTICO: Treinar Soft Skills + Processo'
      -- 2. Problema só de Nota (Comportamental)
      WHEN media_nota_satisfacao < 2.8 THEN '2. Treinar Soft Skills (Foco em Empatia)'
      -- 3. Problema só de Tempo (Técnico)
      WHEN tma_medio_seg > 1200 THEN '3. Treinar Processo (Foco em Agilidade)'
      ELSE '4. Performance OK'
    END AS status_capacitacao
  FROM
    medalhao_case.gold.view_comparativo_atendente_motivo
  WHERE
    total_chamados >= 10 -- Ignora quem atendeu pouco para não sujar o gráfico
)

SELECT
  status_capacitacao,
  COUNT(*) AS quantidade_casos
FROM
  classificacao
GROUP BY
  status_capacitacao
ORDER BY
  status_capacitacao ASC

In [0]:
%sql
SELECT
  nome_atendente,
  nivel_atendimento,
  nome_motivo, -- Onde ele tem dificuldade
  total_chamados,
  tma_medio_seg,
  media_nota_satisfacao,
  taxa_resolucao_percentual,
  
  CASE
    WHEN media_nota_satisfacao < 2.8 AND tma_medio_seg > 1200 THEN 'TREINAMENTO INTENSIVO (Crítico)'
    WHEN media_nota_satisfacao < 2.8 THEN 'Treinar Abordagem ao Cliente'
    WHEN tma_medio_seg > 1200 THEN 'Treinar Sistema/Processo'
    ELSE 'Manter'
  END AS acao_recomendada

FROM
  medalhao_case.gold.view_comparativo_atendente_motivo

WHERE
  total_chamados >= 10 -- Filtro de relevância estatística
  AND (media_nota_satisfacao < 2.8 OR tma_medio_seg > 1200) -- Mostra só quem tem problema

ORDER BY
  media_nota_satisfacao ASC, -- Piores notas primeiro
  tma_medio_seg DESC

In [0]:
%sql
SELECT
  nivel_atendimento,
  criticidade AS criticidade_motivo,
  
  -- Volume e Custo
  SUM(total_chamados) AS volume_chamados,
  CONCAT('R$ ', FORMAT_NUMBER(SUM(custo_total_gerado), 2)) AS custo_total_acumulado,
  
  -- Performance do Grupo
  ROUND(AVG(taxa_resolucao_percentual), 1) AS resolucao_media_grupo,

  -- Diagnóstico Automático de Eficiência
  CASE
    -- Desperdício de Dinheiro: Especialista fazendo trabalho fácil
    WHEN nivel_atendimento = 'NÍVEL 2' AND criticidade = 'Baixa' THEN 'ALERTA DE CUSTO: Especialista em Demanda Simples (Automação Possível)'
    
    -- Risco de Qualidade: Junior fazendo trabalho difícil
    WHEN nivel_atendimento = 'NÍVEL 1' AND criticidade = 'Alta' THEN 'ALERTA DE QUALIDADE: Junior em Demanda Complexa (Baixa Resolução)'
    
    ELSE 'Alocação Correta'
  END AS diagnostico_operacional

FROM
  medalhao_case.gold.view_comparativo_atendente_motivo

GROUP BY
  nivel_atendimento,
  criticidade

ORDER BY
  diagnostico_operacional DESC -- Mostra os alertas primeiro

### View Canal

In [0]:
%sql
create or replace view medalhao_case.gold.view_fcr_por_canal as
with base_cliente_canal as (
    select
        f.fk_canal,
        f.fk_cliente,
        count(distinct f.id_chamado) as qtde_chamados_cliente_canal
    from medalhao_case.gold.ft_chamados_atendimento f
    group by
        f.fk_canal,
        f.fk_cliente
),
cliente_canal_agregado as (
    select
        fk_canal,
        qtde_chamados_cliente_canal,
        case
            when qtde_chamados_cliente_canal >= 2 then 1
            else 0
        end as flag_recontato
    from base_cliente_canal
)
select
    c.nome_canal as canal,
    c.tipo_atendimento as tipo_canal,
    count(*) as total_clientes,
    sum(qtde_chamados_cliente_canal) as qtde_total_chamados,
    sum(flag_recontato)   as qtde_clientes_recontato,
    round(sum(flag_recontato) * 100.0 / count(*), 2) as perc_prob_recontato,
    round(100.0 - (sum(flag_recontato) * 100.0 / count(*)), 2) as fcr_percent
from cliente_canal_agregado b
left join medalhao_case.gold.dim_canal c
    on b.fk_canal = c.nome_canal
group by
    c.nome_canal,
    c.tipo_atendimento;

select *
from medalhao_case.gold.view_fcr_por_canal
order by fcr_percent desc;


In [0]:
%sql
create or replace view medalhao_case.gold.view_custo_resolucao_canal as
select
    f.fk_canal as canal,
    canal.tipo_atendimento as tipo_canal,
    count(*) as qtde_chamados,
    round(
        sum(case when f.atendimento_resolvido = 'Sim' then 1 else 0 end)
        * 100.0 / count(*),2) as taxa_resolucao_percent,
    round(avg(f.custo_atendimento), 2) as custo_medio_chamado,
    round(sum(f.custo_atendimento), 2) as custo_total,
    round(avg(f.tempo_espera_seg), 0) as tme_medio_seg,
    round(avg(f.tempo_atendimento_seg), 0) as tma_medio_seg,
    round(avg(f.nota_atendimento), 2) as nota_media_atendimento
from medalhao_case.gold.ft_chamados_atendimento f
left join medalhao_case.gold.dim_canal canal
    on f.fk_canal = canal.nome_canal
group by
    f.fk_canal,
    canal.tipo_atendimento;

select *
from medalhao_case.gold.view_custo_resolucao_canal
order by taxa_resolucao_percent desc;
